In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Создать Dataset для загрузки данных (sklearn.datasets.fetch_california_housing)

2. Обернуть его в Dataloader

3. Написать архитектуру сети, которая предсказывает стоимость недвижимости. Сеть должна включать   BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)

4. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели
   train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
import torchvision.transforms as transforms

pd.set_option('display.float_format', lambda x: '%0.4f' % x)
np.set_printoptions(suppress=True)

In [3]:
california_housing = fetch_california_housing(as_frame=True)

In [4]:
print(california_housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [5]:
df = california_housing.frame
df.head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0000,6.9841,1.0238,322.0000,2.5556,37.8800,-122.2300,4.5260
1,8.3014,21.0000,6.2381,0.9719,2401.0000,2.1098,37.8600,-122.2200,3.5850
2,7.2574,52.0000,8.2881,1.0734,496.0000,2.8023,37.8500,-122.2400,3.5210
3,5.6431,52.0000,5.8174,1.0731,558.0000,2.5479,37.8500,-122.2500,3.4130
4,3.8462,52.0000,6.2819,1.0811,565.0000,2.1815,37.8500,-122.2500,3.4220
5,4.0368,52.0000,4.7617,1.1036,413.0000,2.1399,37.8500,-122.2500,2.6970
6,3.6591,52.0000,4.9319,0.9514,1094.0000,2.1284,37.8400,-122.2500,2.9920
7,3.1200,52.0000,4.7975,1.0618,1157.0000,1.7883,37.8400,-122.2500,2.4140
8,2.0804,42.0000,4.2941,1.1176,1206.0000,2.0269,37.8400,-122.2600,2.2670
9,3.6912,52.0000,4.9706,0.9902,1551.0000,2.1723,37.8400,-122.2500,2.6110


In [6]:
df.shape

(20640, 9)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [8]:
df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.0000,20640.0000,20640.0000,20640.0000,20640.0000,20640.0000,20640.0000,20640.0000,20640.0000
mean,3.8707,28.6395,5.4290,1.0967,1425.4767,3.0707,35.6319,-119.5697,2.0686
std,1.8998,12.5856,2.4742,0.4739,1132.4621,10.3860,2.1360,2.0035,1.1540
min,0.4999,1.0000,0.8462,0.3333,3.0000,0.6923,32.5400,-124.3500,0.1500
25%,2.5634,18.0000,4.4407,1.0061,787.0000,2.4297,33.9300,-121.8000,1.1960
50%,3.5348,29.0000,5.2291,1.0488,1166.0000,2.8181,34.2600,-118.4900,1.7970
75%,4.7432,37.0000,6.0524,1.0995,1725.0000,3.2823,37.7100,-118.0100,2.6472
max,15.0001,52.0000,141.9091,34.0667,35682.0000,1243.3333,41.9500,-114.3100,5.0000


In [9]:
train_dataset, test_dataset = train_test_split(df, test_size=0.25, random_state=13)

In [10]:
print(f'Train_dataset shape: {train_dataset.shape}\nTest_dataset shape: {test_dataset.shape}')

Train_dataset shape: (15480, 9)
Test_dataset shape: (5160, 9)


In [11]:
class CaliforniaHousingDataset(torch.utils.data.Dataset):
    def __init__(self, init_dataset):
        self._base_dataset = init_dataset

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, index):
        features = np.asarray(self._base_dataset)[index][:-1]
        target = np.asarray(self._base_dataset)[index][-1]
        return features, target

In [12]:
california_housing_train_dataset = CaliforniaHousingDataset(train_dataset)

In [13]:
california_housing_train_dataset[0]

(array([   3.5174    ,   36.        ,    4.54794521,    1.09436834,
        1357.        ,    2.06544901,   34.21      , -118.23      ]),
 2.68)

In [14]:
train_loader = torch.utils.data.DataLoader(california_housing_train_dataset, batch_size=8, shuffle=True)
train_loader

In [15]:
california_housing_test_dataset = CaliforniaHousingDataset(test_dataset)

In [16]:
california_housing_test_dataset[0]

(array([   4.1528    ,   15.        ,    6.19327731,    0.98739496,
         768.        ,    3.22689076,   35.34      , -119.08      ]),
 1.301)

In [17]:
test_loader = torch.utils.data.DataLoader(california_housing_test_dataset, batch_size=8, shuffle=False)
test_loader

In [18]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class LinearRegression(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LinearRegression, self).__init__()
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = Perceptron(hidden_dim, 1, "relu")
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.dp(x)
        x = self.bn(x)
        x = self.fc2(x)
        return x

In [19]:
net = LinearRegression(8, 1)
criterion = nn.MSELoss()

### SGD

In [20]:
%%time

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in tqdm(range(10)):  
    running_loss, running_items = 0.0, 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(torch.float), data[1].to(torch.float)
        labels = labels.reshape(8, 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Loss: {running_loss / running_items:.3f}. ')
            
            running_loss, running_items = 0.0, 0.0

            net.train()
        
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

Loss: 1.112. 
Loss: 0.680. 
Loss: 0.694. 
Loss: 0.704. 
Loss: 0.700. 
Loss: 0.710. 


 10%|█         | 1/10 [00:01<00:17,  1.89s/it]

Loss: 0.716. 
Loss: 0.643. 
Loss: 0.706. 
Loss: 0.696. 
Loss: 0.704. 
Loss: 0.685. 
Loss: 0.700. 


 20%|██        | 2/10 [00:03<00:14,  1.78s/it]

...

Loss: 0.709. 
Loss: 0.889. 
Loss: 0.703. 
Loss: 0.713. 
Loss: 0.666. 
Loss: 0.693. 
Loss: 0.695. 


 90%|█████████ | 9/10 [00:15<00:01,  1.72s/it]

Loss: 0.711. 
Loss: 0.551. 
Loss: 0.711. 
Loss: 0.679. 
Loss: 0.706. 
Loss: 0.699. 
Loss: 0.710. 


100%|██████████| 10/10 [00:17<00:00,  1.73s/it]

Loss: 0.686. 
Training is finished!
CPU times: user 17 s, sys: 112 ms, total: 17.2 s
Wall time: 17.3 s


### RMSProp

In [21]:
%%time

optimizer = torch.optim.RMSprop(net.parameters(), lr=0.1, alpha=0.99)

for epoch in tqdm(range(10)):  
    running_loss, running_items = 0.0, 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(torch.float), data[1].to(torch.float)
        labels = labels.reshape(8, 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Loss: {running_loss / running_items:.3f}. ')
            
            running_loss, running_items = 0.0, 0.0

            net.train()
        
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

Loss: 0.578. 
Loss: 0.707. 
Loss: 0.666. 
Loss: 0.708. 
Loss: 0.699. 
Loss: 0.689. 


 10%|█         | 1/10 [00:02<00:19,  2.13s/it]

Loss: 0.707. 
Loss: 0.630. 
Loss: 0.719. 
Loss: 0.699. 
Loss: 0.687. 
Loss: 0.700. 
Loss: 0.705. 


 20%|██        | 2/10 [00:03<00:15,  1.96s/it]

...

Loss: 0.682. 
Loss: 0.627. 
Loss: 0.710. 
Loss: 0.689. 
Loss: 0.692. 
Loss: 0.702. 
Loss: 0.718. 


 90%|█████████ | 9/10 [00:16<00:01,  1.86s/it]

Loss: 0.690. 
Loss: 0.776. 
Loss: 0.695. 
Loss: 0.687. 
Loss: 0.693. 
Loss: 0.692. 
Loss: 0.714. 


100%|██████████| 10/10 [00:18<00:00,  1.87s/it]

Loss: 0.704. 
Training is finished!
CPU times: user 18.6 s, sys: 43.3 ms, total: 18.6 s
Wall time: 18.7 s


### Adam

In [22]:
%%time

optimizer = torch.optim.Adam(net.parameters(), lr=0.1)

for epoch in tqdm(range(10)):  
    running_loss, running_items = 0.0, 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(torch.float), data[1].to(torch.float)
        labels = labels.reshape(8, 1)

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)
        
        # выводим статистику о процессе обучения
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            net.eval()

            print(f'Loss: {running_loss / running_items:.3f}. ')
            
            running_loss, running_items = 0.0, 0.0

            net.train()
        
print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

Loss: 0.958. 
Loss: 0.685. 
Loss: 0.701. 
Loss: 0.723. 
Loss: 0.699. 
Loss: 0.721. 


 10%|█         | 1/10 [00:01<00:17,  2.00s/it]

Loss: 0.677. 
Loss: 0.744. 
Loss: 0.671. 
Loss: 0.735. 
Loss: 0.697. 
Loss: 0.681. 
Loss: 0.697. 


 20%|██        | 2/10 [00:04<00:17,  2.15s/it]

...

Loss: 0.704. 
Loss: 0.549. 
Loss: 0.706. 
Loss: 0.713. 
Loss: 0.701. 
Loss: 0.692. 
Loss: 0.694. 


 90%|█████████ | 9/10 [00:19<00:02,  2.06s/it]

Loss: 0.684. 
Loss: 0.515. 
Loss: 0.690. 
Loss: 0.694. 
Loss: 0.713. 
Loss: 0.670. 
Loss: 0.708. 


100%|██████████| 10/10 [00:21<00:00,  2.10s/it]

Loss: 0.727. 
Training is finished!
CPU times: user 20.8 s, sys: 55.1 ms, total: 20.9 s
Wall time: 21 s


### Вывод: 

Как мы видим SGD отработал быстрее всех, однако Adam был самый медленный. Показатели Loss у SGD и RMSProp идентичны 0.168, а у Adam Loss значительно хуже 0.170 также он несколько медленнее. Поэтому оптимальным для выполнения данной задачи считаю SGD